### How to query the Bedrock agent example

In [11]:
import uuid
import json
import streamlit as st
from botocore.config import Config
from boto3.session import Session

In [4]:
# SDK init
boto_session = Session().client("bedrock-agent-runtime", config=Config(read_timeout=600))

In [ ]:
def invoke_agent(agent_id, agent_alias_id, input_text, trace):
    """_summary_

    Args:
        agent_id (_type_): _description_
        agent_alias_id (_type_): _description_
        input_text (_type_): _description_
        trace (_type_): _description_

    Raises:
        Exception: _description_

    Returns:
        _type_: _description_
    """

    # Var init
    response_text = ""
    trace_text = ""
    step = 0

    response = boto_session.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id,
        sessionId=str(uuid.uuid1()),
        enableTrace=True,
    )

    try:
        for event in response["completion"]:
            if "chunk" in event:

                data = event["chunk"]["bytes"]
                response_text = data.decode("utf8")

            elif "trace" in event:

                trace_obj = event["trace"]["trace"]

                if "orchestrationTrace" in trace_obj:

                    trace_dump = json.dumps(
                        trace_obj["orchestrationTrace"], indent=2
                    )

                    if "rationale" in trace_obj["orchestrationTrace"]:

                        step += 1
                        trace_text += f'\n\n\n---------- Step {step} ----------\n\n\n{trace_obj["orchestrationTrace"]["rationale"]["text"]}\n\n\n'
                        # trace.markdown(f'\n\n\n---------- Step {step} ----------\n\n\n{trace_obj["orchestrationTrace"]["rationale"]["text"]}\n\n\n')

                    elif (
                        "modelInvocationInput"
                        not in trace_obj["orchestrationTrace"]
                    ):

                        trace_text += "\n\n\n" + trace_dump + "\n\n\n"
                        # trace.markdown("\n\n\n" + trace_dump + "\n\n\n")

                elif "failureTrace" in trace_obj:

                    trace_text += "\n\n\n" + trace_dump + "\n\n\n"
                    # trace.markdown("\n\n\n" + trace_dump + "\n\n\n")

                elif "postProcessingTrace" in trace_obj:

                    step += 1
                    trace_text += f"\n\n\n---------- Step {step} ----------\n\n\n{json.dumps(trace_obj['postProcessingTrace']['modelInvocationOutput']['parsedResponse']['text'], indent=2)}\n\n\n"
                    # trace.markdown(f"\n\n\n---------- Step {step} ----------\n\n\n{json.dumps(trace_obj['postProcessingTrace']['modelInvocationOutput']['parsedResponse']['text'], indent=2)}\n\n\n")

    except Exception as e:
        trace_text += str(e)
        # trace.markdown(str(e))
        raise Exception("unexpected event.", e)

    return response_text, trace_text

In [16]:
prompt = "Hello"
response_text, trace_text = invoke_agent(agent_id="YQNH56DV9T", agent_alias_id="7MUST2FRIK", input_text=prompt, trace=None)
print(response_text)

Hello! Welcome to our department store. I'm the virtual assistant here to help you with any questions you might have about our products or to assist you with making a purchase. How can I help you today?
